In [1]:
import numpy as np 
import pandas as pd 

import json
import time
from pandas.io.json import json_normalize
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.metrics import explained_variance_score, roc_auc_score, precision_recall_curve, roc_curve, average_precision_score,accuracy_score
from sklearn import model_selection, preprocessing, metrics
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, cross_val_score, cross_validate
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

from mpl_toolkits.basemap import Basemap

import gc
import os
print(os.listdir("."))

['.DS_Store', 'input', 'Assignment-1.pdf', '.ipynb_checkpoints', 'electricity-demand-prediction.ipynb']


In [2]:
in_file_name = "Home1_yr1.csv"
out_file_name = "home1_out.csv"

In [3]:
house_data = pd.read_csv("./input/" + in_file_name, low_memory=False, nrows=None)
house_data.shape

(35039, 1)

In [4]:
house_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35039 entries, 0 to 35038
Data columns (total 1 columns):
0.65018    35039 non-null float64
dtypes: float64(1)
memory usage: 273.8 KB


In [5]:
house_data.describe()

,0.65018
count,35039.000000
mean,1.327227
std,1.399034
min,0.015124
25%,0.315675
50%,0.722890
75%,1.988550
max,15.500000


In [8]:

epochTime = 1417392000

def get_time_stamp(row):
    global epochTime
    epochTime+=900
    return pd.to_datetime(epochTime,unit='s')

house_data['time_stamp'] = house_data.apply (lambda row: get_time_stamp(row),axis=1)


In [10]:
house_data.head(100)

,0.65018,time_stamp
0,1.45400,2014-12-01 00:15:00
1,0.72971,2014-12-01 00:30:00
2,3.10750,2014-12-01 00:45:00
3,0.63572,2014-12-01 01:00:00
4,0.69720,2014-12-01 01:15:00
5,1.50610,2014-12-01 01:30:00
6,0.85858,2014-12-01 01:45:00
7,1.61320,2014-12-01 02:00:00
8,0.66075,2014-12-01 02:15:00
9,0.50952,2014-12-01 02:30:00
